# Aliasing

The previous section introduced uniform sampling, which allows us to represent a continuous signal $x(t)$ by a discrete sequence of sample values $x[n]$.

In this section, we'll see that this idea comes with some restrictions.


In [7]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cycler
import matplotlib.path as path
import matplotlib.patches as patches

import warnings
warnings.simplefilter('ignore')

from matplotlib.animation import FuncAnimation
import matplotlib.style
matplotlib.style.use('bauhaus')

np.random.seed(20200101)

colors = [_['color'] for _ in list(matplotlib.rcParams['axes.prop_cycle'])]

## What is aliasing?

**Aliasing** is the name we give to the phenomenon when two distinct continuous signals $x_1(t)$ and $x_2(t)$ produce the same sequence of sample values $x[n]$ when sampled at a fixed rate $f_s$.

More specifically, we usually think of aliasing in terms of pure (sinusoidal) tones $x(t) = A \cdot \cos\left(2\pi \cdot f \cdot t + \phi\right)$.
Given a sampling rate $f_s$, two frequencies $f$ and $f'$ are **aliases** of each other if for some integer $k$,
$$
f' = f + k \cdot f_s.
$$
Or, in words, frequency $f'$ is $f$ plus some whole number multiples of the sampling rate $f_s$.  This rule for relating aliased frequencies is known as the *aliasing equation*.

The figure below illustrates this effect: for any $f$ that we choose, once a sampling rate $f_s$ is chosen, there are infinitely many frequencies that produce the exact same samples.

In [62]:
%%capture
## Animation of aliasing

frame_rate = 20

# 1 second fade-in fade-out for each alias
anim_duration = 12
n_frames = anim_duration * frame_rate

sched_f0 = np.zeros(n_frames)
sched_f0[-frame_rate:] = np.linspace(0, 1, num=frame_rate, endpoint=False)
sched_f0[:2*frame_rate] = 1
sched_f0[2*frame_rate:3*frame_rate] = np.linspace(1, 0, num=frame_rate, endpoint=False)

sched_f1 = np.zeros(n_frames)
sched_f1[2*frame_rate:3*frame_rate] = np.linspace(0, 1, num=frame_rate, endpoint=False)
sched_f1[3*frame_rate:5*frame_rate] = 1
sched_f1[5*frame_rate:6*frame_rate] = np.linspace(1, 0, num=frame_rate, endpoint=False)

sched_f2 = np.zeros(n_frames)
sched_f2[5*frame_rate:6*frame_rate] = np.linspace(0, 1, num=frame_rate, endpoint=False)
sched_f2[6*frame_rate:8*frame_rate] = 1
sched_f2[8*frame_rate:9*frame_rate] = np.linspace(1, 0, num=frame_rate, endpoint=False)

sched_f3 = np.zeros(n_frames)
sched_f3[8*frame_rate:9*frame_rate] = np.linspace(0, 1, num=frame_rate, endpoint=False)
sched_f3[9*frame_rate:11*frame_rate] = 1
sched_f3[11*frame_rate:] = np.linspace(1, 0, num=frame_rate, endpoint=False)

sched_f0 **=0.5
sched_f1 **=0.5
sched_f2 **=0.5
sched_f3 **=0.5

fig = plt.figure(figsize=(8, 6))

fs_real = 1000
fs = 5
duration = 2

f0 = 1

f1 = f0 + fs
f2 = f0 + 2 * fs
f3 = f0 - fs

# Plot the continuous time curves
t = np.linspace(0, duration, num=duration * fs_real, endpoint=False)
x0 = np.cos(2 * np.pi * f0 * t)
x1 = np.cos(2 * np.pi * f1 * t)
x2 = np.cos(2 * np.pi * f2 * t)
x3 = np.cos(2 * np.pi * f3 * t)

# Plot the samples
N = int(duration * fs)
xsamp = np.cos(2 * np.pi * f0 * np.arange(N) / fs)
tsamp = np.arange(N) / fs

p0 = plt.plot(t, x0, label=r'$f={}$ Hz'.format(f0), color=colors[0])[0]
p1 = plt.plot(t, x1, label=r'$f={}$ Hz'.format(f1), color=colors[2])[0]
p2 = plt.plot(t, x2, label=r'$f={}$ Hz'.format(f2), color=colors[3])[0]
p3 = plt.plot(t, x3, label=r'$f={}$ Hz'.format(f3), color=colors[4])[0]

plt.plot(tsamp, xsamp, linestyle='', marker='.', color=colors[1], label=r'$x[n]$', zorder=5)

plt.legend(ncol=3, loc='upper right', fontsize='large')
plt.title(r'Aliasing example at $f_s={}$ Hz'.format(fs))
plt.xlabel('Time $t$ [seconds]')

def init():
    return p0, p1, p2, p3

def animate(i):
    # Cross-fade the different curves
    p0.set_alpha(sched_f0[i])
    p1.set_alpha(sched_f1[i])
    p2.set_alpha(sched_f2[i])
    p3.set_alpha(sched_f3[i])
    return p0, p1, p2, p3

animation = FuncAnimation(fig, animate, init_func=init,
                               frames=n_frames, interval=frame_rate, blit=True)

In [63]:
animation

## Why is aliasing a problem?

Aliasing is an unavoidable consequence of digital sampling: there will always be frequencies that look the same after sampling.
The consequence of this fact is that once you've sampled a signal, you may not be able to determine the frequency of the wave that produced the samples you've observed!

We'll see in the next section that the Nyquist-Shannon theorem suggests a resolution to this problem, but for now let's work toward a better understanding of why aliasing occurs.

## Why does aliasing happen?

In the previous section, we sampled a pure tone by using the following equation
$$
x[n] = \cos\left(2\pi \cdot f \cdot \frac{n}{f_s} \right).
$$
$$
\require{cancel}\notag%
\require{color}\notag%
$$
To see why aliasing occurs between frequencies $f$ and $f + \color{red}{k \cdot f_s}$, we can plug the latter into the equation above and see what happens.
The key idea that we'll need is the following identity:

$$
\cos(\theta + 2\pi\cdot m) = \cos(\theta) \;\;\;\text{for any integer } m.
$$

This identity works because if $m$ is an integer, then $2\pi\cdot m$ is a whole number of rotations around the circle (clockwise if $m < 0$, counter-clockwise if $m > 0$).
As a result, adding $2\pi\cdot m$ to any angle $\theta$ leaves you back at $\theta$.

By analogy, you can think of $\theta$ as the minute hand on a clock, and each $m$ counts an hour offset.  Regardless of the current time, adding one hour (or two, or three, counted by $m$) will leave the minute hand in exactly the same place.

### Proof of aliasing

The proof of the aliasing equation uses a bit of algebra to re-arrange the terms of the equation, and then exploits the identiy defined above to simplify the equation.
We start with the definition for the sequence generated by sampling a pure tone at $f + \color{red}{k\cdot f_s}$, and work our way toward the sequence generated by $f$.
We'll ignore amplitude $A$ and phase offset $\phi$ to avoid cluttering the notation, but the argument goes through just as well when those features are included.

$$
\begin{align*}
\cos\left(2\pi \cdot \left(f + \color{red}{k\cdot f_s}\right) \cdot \frac{n}{f_s} \right)
&= 
\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}  + \color{red}{2\pi \cdot k\cdot f_s \cdot \frac{n}{f_s}} \right) &\text{Distribute multiplication}\\
&= 
\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}  + \color{red}{2\pi \cdot k\cdot \cancel{f_s} \cdot \frac{n}{\cancel{f_s}}} \right)&\text{Cancel } f_s\\
&= 
\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}  + \color{red}{2\pi \cdot k\cdot n} \right)\\
&= 
\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}  + \color{red}{\cancel{2\pi \cdot k\cdot n}} \right)& k\cdot n \text{ is an integer}\\
&=\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}\right)
\end{align*}
$$

This shows that the two frequencies $f$ and $f + \color{red}{k\cdot f_s}$ produce the same sequence of samples, regardless of $k$! (But definitely depending on $f_s$.)

If we were to listen to sampled tones at these frequencies, we shouldn't be able to tell them apart.
Let's test that hypothesis!

## Example: aliased tones

The following code example generates two tones at aliasing frequencies, but is otherwise very similar to the example in the previous section.
Try it out!  The two sequences of samples will be numerically identical, and therefore sound identical.

Here, we just used $k=1$, but any integer $k$ will produce the same results.

In [5]:
import numpy as np
from IPython.display import display, Audio

# Sampling at 8 KHz
fs = 8000

# We'll make a pure tone at 220, and at 220 + 8000 = 8220
f0_original = 220
f0_aliased = f0_original + fs

# 2 seconds of audio should be plenty
duration = 2

# How many samples is 2 seconds?
N = int(duration * fs)

# Generate the sample positions
n = np.arange(N)

# Construct the first signal
x_original = np.cos(2 * np.pi * f0_original * n / fs)

# And the aliased signal
x_aliased = np.cos(2 * np.pi * f0_aliased * n / fs)

# Let's hear them both!
print("fs = {} Hz, f0 = {} Hz".format(fs, f0_original))
display(Audio(data=x_original, rate=fs))

print("fs = {} Hz, f0 aliased = {} Hz".format(fs, f0_aliased))
display(Audio(data=x_aliased, rate=fs))

fs = 8000 Hz, f0 = 220 Hz


fs = 8000 Hz, f0 aliased = 8220 Hz


## Summary

